In [68]:
%pip install numpy pandas nltk spacy regex contractions scikit-learn 
 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
import os
import pandas as pd
import re
import spacy
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

#### Loading Dataset


In [70]:

file_path = "raw_dataset.csv"
df_r=pd.read_csv(file_path)


In [71]:
df_r = df_r.drop_duplicates(subset='review').reset_index(drop=True)

In [72]:
df_r['label'].value_counts()

label
OR    20215
CG    20197
Name: count, dtype: int64

In [73]:
df_r['label'] = df_r['label'].map({'CG': 0, 'OR': 1})

#### Pre Pre-processing


In [ ]:
def capital_letter_ratio(text):
    if not isinstance(text, str):
        return 0.0

    letters = [c for c in text if c.isalpha()]
    if len(letters) == 0:
        return 0.0

    capital_letters = [c for c in letters if c.isupper()]
    return len(capital_letters) / len(letters)
df_r['capital_ratio'] = df_r['review'].apply(capital_letter_ratio)

In [ ]:
def punctuation_ratio(review):
    if not isinstance(review, str) or len(review) == 0:
        return 0.0

    punct_count = len(re.findall(r"[^\w\s]", review))
    return punct_count / len(review)
df_r['punctuation_ratio'] = df_r['review'].apply(punctuation_ratio)

In [ ]:
df_r['text_length'] = df_r['review'].apply(lambda x: len(str(x)))


#### Pre-processing


In [ ]:
# contraction expansion
import contractions
def expand_contractions(review):
    if pd.isna(review):
        return ""
    return contractions.fix(review)
df_r['expanded_text'] = df_r['review'].apply(expand_contractions)

In [ ]:
# cleaning text - lowercase, remove url, html tags, punctiation, whitespaces
def clean_text(review):
    if pd.isna(review):
        return ""
    
    review = review.lower()
    
    review = re.sub(r'http\S+|www\S+', '', review)
    review = re.sub(r'<.*?>', '', review)
    
    # remove punctuation (letters + spaces only)
    review = re.sub(r'[^a-z\s]', '', review)
    
    review = re.sub(r'\s+', ' ', review).strip()
    
    return review
df_r['clean_text'] = df_r['expanded_text'].apply(clean_text)

In [ ]:
# lemmatization
# nltk resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def adjective_ratio(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0
    
    tokens = word_tokenize(text)        
    pos_tags = pos_tag(tokens)          
    
    adj_count = sum(1 for word, tag in pos_tags if tag.startswith('JJ'))
    total_words = len(tokens)
    
    return adj_count / total_words if total_words > 0 else 0
df_r['adjective_ratio'] = df_r['clean_text'].apply(adjective_ratio)

In [ ]:
sia = SentimentIntensityAnalyzer()
def sentiment_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0
    
    # Compound score ranges from -1 (very negative) to +1 (very positive)
    return sia.polarity_scores(text)['compound']
df_r['sentiment_score'] = df_r['clean_text'].apply(sentiment_score)

In [ ]:

nlp = spacy.load("en_core_web_sm")
import pandas as pd

def lemmatize_text(review):
    if pd.isna(review) or review == "":
        return ""
    
    doc = nlp(review)
    
    lemmatized_words = [
        token.lemma_
        for token in doc
        if not token.is_space
    ]
    
    return " ".join(lemmatized_words)
df_r['review'] = df_r['clean_text'].apply(lemmatize_text)

#### Preprocessed dataset

In [57]:
df_r.columns

Index(['category', 'rating', 'label', 'text_', 'capital_ratio',
       'punctuation_ratio', 'is_excessive_punctuation', 'expanded_text',
       'clean_text', 'adjective_ratio', 'sentiment_score', 'text_length',
       'review'],
      dtype='object')

In [ ]:
pre_df = df_r[
    ['review','clean_text', 'review', 'rating','label', 'reviewlength',
     'capital_ratio', 'punctuation_ratio', 'is_excessive_punctuation',
     'adjective_ratio', 'sentiment_score']
]

# Save as CSV
pre_df.to_csv("preprocessed_dataset.csv", index=False)
print("Preprocessed dataset saved as CSV!")